# Longformer
- potential to dos; Data_Iterator
- [Huggingface datasets](https://drive.google.com/drive/folders/1UQfl6oXyYt4Eepudmgi6A9xMAkqBuaHf)
- Key functions:
- HuggingFace dataset output: [/data/tab/longformer]
    - [/data/tab/longformer](https://drive.google.com/drive/folders/1UQfl6oXyYt4Eepudmgi6A9xMAkqBuaHf) - for multiclassification; label names = ner_labels; mask_labels
    - [/data/tab/longformer_mask](https://drive.google.com/drive/folders/1bgkTuZ428fLdnFrtq0BWJcTBT3lpNXbK) - single classification; labels
    - [/data/tab/longformer_ner](https://drive.google.com/drive/folders/1M8KiTXhpdkiMzJRqcLX0X7KY0dCbqw3t) - single classification; labels

In [1]:
!pip install -q transformers
!pip install -q datasets
!pip install -q evaluate
!pip install -q seqeval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
# generic
import os
import numpy as np
# from pprint import pprint

# ml
from datasets import load_dataset, load_from_disk, Dataset
from transformers import LongformerForTokenClassification, LongformerTokenizerFast, Trainer, TrainingArguments, EarlyStoppingCallback
import evaluate
import torch
import torch.nn as nn
from seqeval.metrics import classification_report, precision_score, recall_score, f1_score, accuracy_score
# from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
# import tensorflow as tf
# from tensorflow import keras

In [3]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [4]:
# use for vertex ai / google cloud
# from google.cloud import storage

# client = storage.Client()
# bucket_name = 'w266-project'
# bucket = client.get_bucket(bucket_name)
# path = f'gs://{bucket_name}'
# vertex_path = '/content'

# use for google collab
from google.colab import drive

drive.mount('/content/drive')
path = '/content/drive/MyDrive/Colab Notebooks/266 Project'

Mounted at /content/drive


In [5]:
def print_version(library_name):
    try:
        lib = __import__(library_name)
        version = getattr(lib, '__version__', 'Version number not found')
        print(f"{library_name} version: {version}")
    except ImportError:
        print(f"{library_name} not installed.")
    except Exception as e:
        print(f"An error occurred: {e}")

print_version('transformers')
print_version('tensorflow')
print_version('keras')

transformers version: 4.46.3
tensorflow version: 2.17.1
keras version: 3.5.0


In [6]:
# global variables
model_checkpoint = 'allenai/longformer-base-4096'
tokenizer = LongformerTokenizerFast.from_pretrained(model_checkpoint, add_prefix_space=True)

task = 'both' # DON'T CHANGE!
size = 'mini' # testing, mini, full

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

# Longformer Model

## Functions

In [7]:
# general functions
def select_data(split, task, size):
    """
    Loads the appropriate dataset per folder structure here: https://drive.google.com/drive/folders/1C3h3rXdbr9nVAC3_G_I-72DfKNiDU_Pa
    Input:
        Split: ['train', 'val', 'test']
        Task: ['ner', 'mask', 'both']
        Size: ['testing', 'mini', 'full']
    Returns:
        Huggingface dataset
    """
    if split not in ['train', 'val', 'test']:
        raise ValueError("Split value must be in ['train', 'val', 'test']")
    if task not in ['ner', 'mask', 'both']:
        raise ValueError("Task value must be in ['ner', 'mask', 'both']")
    if size not in ['testing', 'mini', 'full']:
        raise ValueError("Size value must be in ['testing', 'mini', 'full']")

    path_label = {'both': 'longformer', 'ner': 'longformer_ner', 'mask': 'longformer_mask'}
    # path_label = {'both': 'longformer', 'ner': 'longformer_ner', 'mask': 'longformer_4096'}

    if size == 'testing':
        ds = load_from_disk(f'{path}/data/tab/{path_label[task]}/lf_{split}_testing')
    if size == 'mini':
        if split == 'train':
            ds = load_from_disk(f'{path}/data/tab/{path_label[task]}/lf_{split}_400')
        else:
            ds = load_from_disk(f'{path}/data/tab/{path_label[task]}/lf_{split}_50')
    if size == 'full':
        ds = load_from_disk(f'{path}/data/tab/{path_label[task]}/lf_{split}')

    return ds


# def create_dataset(split, task, size):
#     """Creates appropriate dataset depending on training objective.
#     Input:
#         dataset = use load_from_disk(<path>)
#     Output:
#         returns dataset for training
#     """

#     if task == 'both':
#         labels = ['ner_labels', 'mask_labels']
#     else:
#         labels = ['labels']

#     ds = select_data(split=split, task=task, size=size)

#     data_collator = DataCollatorForTokenClassification(tokenizer,
#                                                        padding='max_length',
#                                                        max_length=4096,
#                                                        return_tensors='np')

#     data_set = ds['train'].to_tf_dataset(
#         columns=['input_ids', 'attention_mask'],
#         label_cols=labels,
#         shuffle=True,
#         batch_size=16,
#         collate_fn=data_collator
#     )

#     return data_set

def update_attention_weights(dataset, weights):
    ner_classes = ['O', 'B-PERSON', 'I-PERSON', 'B-CODE', 'I-CODE', 'B-LOC', 'I-LOC', 'B-ORG', 'I-ORG',
                'B-DEM', 'I-DEM', 'B-DATETIME', 'I-DATETIME', 'B-QUANTITY', 'I-QUANTITY', 'B-MISC', 'I-MISC']

    ner_weights = {}
    for i, ner in enumerate(ner_classes):
        ner_weights[i] = weights.get(ner, weights.get('OTHER', 0))

    ner_labels = dataset['train']['ner_labels']
    attention_mask = dataset['train']['attention_mask']
    mask_adjustments = []
    new_attention_masks = []
    for i, sample in enumerate(ner_labels):
        sample_adjust = []
        cls_count = 0
        for s in sample:
            if cls_count < 2:
                sample_adjust.append(1)
                cls_count += 1
            elif s < 0:
                sample_adjust.append(0)
            else:
                sample_adjust.append(ner_weights[s])
        mask_adjustments.append(sample_adjust)
        new_mask = list(np.array(attention_mask[i]) * np.array(sample_adjust))
        new_attention_masks.append(new_mask)

    return mask_adjustments, new_attention_masks

def create_dataset_attention(ds, attention_masks):
    new_ds_dict = {'id': ds['train']['id'],
          'input_ids': ds['train']['input_ids'],
          'attention_mask': attention_masks,
          'labels': ds['train']['mask_labels']}
    new_ds = Dataset.from_dict(new_ds_dict)

    return new_ds

In [8]:
# NOT IN USE FOR THIS
# class LongformerPreAttention(nn.Module):
#     # def __init__(self, ner_embed_dim, max_seq_len=4096):
#     def __init__(self):
#         super(LongformerConcat, self).__init__()
#         # Load the base Longformer model
#         self.longformer = LongformerModel.from_pretrained("allenai/longformer-base-4096")
#         hidden_size = self.longformer.config.hidden_size

#         # Define ner input layer
#         # self.ner_ids = nn.Linear(ner_dim, hidden_size)
#         # self.ner_embedding = nn.Embedding(7, ner_embed_dim)
#         # self.concat = nn.Linear(self.hidden_size + ner_embed_dim, self.hidden_size)

#         # Define the classification head
#         self.classifier = nn.Sequential(
#             nn.Dropout(0.1),
#             nn.Linear(hidden_size, 7)  # Hidden size of Longformer -> num_labels
#         )

#     # def forward(self, input_ids, attention_mask, ner_ids):
#     def forward(self, input_ids, attention_mask):
#         # Pass inputs through the base Longformer model
#         lf_outputs = self.longformer(
#             input_ids=input_ids,
#             attention_mask=attention_mask,
#         )
#         lf_hidden_states = lf_outputs.last_hidden_state  # Shape: (batch_size, hidden_size)

#         # begin custom model
#         # ner_embeds = self.ner_embedding(ner_ids)
#         # concat_embeds = torch.cat((lf_hidden_states, ner_embeds), dim=-1)
#         # projected_embeds = self.concat_project(concat_embeds)

#         # Pass through the classification head
#         # logits = self.classifier(projected_embeds)  # Shape: (batch_size, num_labels)

#         # Optionally calculate loss if labels are provided
#         # loss = None
#         # if labels is not None:
#         #     loss_fn = nn.CrossEntropyLoss(ignore_index=-100)
#         #     loss = loss_fn(logits, labels)

#         # return {"logits": logits, "loss": loss} if loss is not None else {"logits": logits}
#         return lf_hidden_states

# # Example usage
# num_labels = 3  # Assume we are doing 3-class classification
# model = LongformerForCustomClassification(num_labels)

# # Example inputs
# batch_size, seq_length = 2, 512
# input_ids = torch.randint(0, 30522, (batch_size, seq_length))  # Random input IDs
# attention_mask = torch.ones(batch_size, seq_length)  # Attention mask
# global_attention_mask = torch.zeros(batch_size, seq_length)  # Global attention mask
# labels = torch.tensor([0, 2])  # Example labels for 2 samples

# # Forward pass
# outputs = model(input_ids=input_ids, attention_mask=attention_mask, global_attention_mask=global_attention_mask, labels=labels)

# # Logits and loss
# print("Logits:", outputs["logits"])
# print("Loss:", outputs.get("loss"))

In [9]:
# metrics
def compute_metrics(p):
    seqeval = evaluate.load('seqeval')

    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    label_list = ['O', 'B-NO_MASK', 'I-NO_MASK', 'B-DIRECT', 'I-DIRECT', 'B-QUASI', 'I-QUASI']
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels, zero_division=1)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "seqeval_acc": results["overall_accuracy"],
    }

def count_trainable_parameters(model):
    # Get the trainable parameters of the model
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return trainable_params

## Model

### Training

In [10]:
# all ner_tags have weight of 1; O has half weight
ner_attention_weights_equal = {'O': .25,
                               'OTHER': 1}

# high direct vs high quasi
out = .5
direct = 1.5
quasi = 1
ner_attention_weights_weighted_1 = {'O': out,
                                    'B-PERSON': direct,
                                    'I-PERSON': direct,
                                    'B-CODE': direct,
                                    'I-CODE': direct,
                                    'B-LOC': quasi,
                                    'I-LOC': quasi,
                                    'B-ORG': quasi,
                                    'I-ORG': quasi,
                                    'B-DEM': quasi,
                                    'I-DEM': quasi,
                                    'B-DATETIME': quasi,
                                    'I-DATETIME': quasi,
                                    'B-QUANTITY': quasi,
                                    'I-QUANTITY': quasi,
                                    'B-MISC': quasi,
                                    'I-MISC': quasi}

# order of direct + quasi
ner_attention_weights_weighted_2 = {'O': out,
                                    'B-PERSON': 1.4,
                                    'I-PERSON': 1.4,
                                    'B-CODE': 1.4,
                                    'I-CODE': 1.4,
                                    'B-LOC': 1.2,
                                    'I-LOC': 1.2,
                                    'B-ORG': 1,
                                    'I-ORG': 1,
                                    'B-DEM': 1,
                                    'I-DEM': 1,
                                    'B-DATETIME': 1,
                                    'I-DATETIME': 1,
                                    'B-QUANTITY': 1.4,
                                    'I-QUANTITY': 1.4,
                                    'B-MISC': 1,
                                    'I-MISC': 1}

In [21]:
# ds = select_data(split='train', task=task, size=size)
# train_set = create_dataset(split='train', task=task, size=size)
# val_set = create_dataset(split='val', task=task, size=size)

ds_train = select_data(split='train', task=task, size=size)
mask_adjustments, new_attention_masks = update_attention_weights(ds_train, ner_attention_weights_equal)
new_ds_train = create_dataset_attention(ds_train, new_attention_masks)

ds_val = select_data(split='val', task=task, size=size)
mask_adjustments, new_attention_masks = update_attention_weights(ds_val, ner_attention_weights_equal)
new_ds_val = create_dataset_attention(ds_val, new_attention_masks)


In [27]:
model_name = 'pexp4_lp_pa_mini_equal_.25_1'

# possible to implement accerate.utils; auto_find_batch_size
batch_size = 8
num_epochs = 10

# (2.5e-5, 5e-4, 1e-4)
# training_args = TrainingArguments(
#     output_dir=f'{path}/models/{model_name}/results',
#     eval_strategy='epoch',
#     save_strategy='epoch',
#     logging_strategy='epoch',
#     save_total_limit=2,
#     load_best_model_at_end=True,
#     save_only_model=True,
#     metric_for_best_model='eval_loss',
#     greater_is_better=False,
#     learning_rate=1e-4, #
#     num_train_epochs=num_epochs,
#     # lr_scheduler_type='cosine',
#     # lr_scheduler_kwargs={'num_warmup_steps': 50, 'num_training_steps': 50},
#     warmup_ratio=0.1, # only for linear warmup
#     # weight_decay=0.01,
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
#     fp16=True,
#     report_to="none"
# )

training_args = TrainingArguments(
    output_dir=f'{path}/models/{model_name}/results',
    # eval_strategy='epoch',
    # save_strategy='epoch',
    # logging_strategy='epoch',
    # save_total_limit=2,
    # load_best_model_at_end=True,
    # save_only_model=True,
    # metric_for_best_model='eval_loss',
    # greater_is_better=False,
    # learning_rate=1e-4, #
    # num_train_epochs=num_epochs,
    # # lr_scheduler_type='cosine',
    # # lr_scheduler_kwargs={'num_warmup_steps': 50, 'num_training_steps': 50},
    # warmup_ratio=0.1, # only for linear warmup
    # # weight_decay=0.01,
    # per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True,
    report_to="none"
)

In [29]:
f'{path}/models/{model_name}/results/checkpoint-150'

'/content/drive/MyDrive/Colab Notebooks/266 Project/models/pexp4_lp_pa_mini_equal_.25_1/results/checkpoint-150'

In [30]:
# build model
# model = LongformerForTokenClassification.from_pretrained(model_checkpoint, gradient_checkpointing=True, num_labels=7)
model = LongformerForTokenClassification.from_pretrained(f'{path}/models/{model_name}/results/checkpoint-150')

print('trainable parameters:', count_trainable_parameters(model))
# print(model)

trainable parameters: 148074247


In [31]:
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=new_ds_train,
#     eval_dataset=new_ds_val,
#     compute_metrics=compute_metrics,
#     callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
# )

trainer = Trainer(
    model=model,
    args=training_args,
    # train_dataset=new_ds_train,
    # eval_dataset=new_ds_val,
    compute_metrics=compute_metrics#,
    # callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

In [ ]:
trainer.train()

In [ ]:
# save hf/pytorch model
trainer.save_model(f'{path}/models/{model_name}/model')
# did not save tokenizer as already tokenized; load default longformer

In [32]:
trainer.evaluate(eval_dataset=new_ds_train)

{'eval_loss': 0.0757531151175499,
 'eval_model_preparation_time': 0.0067,
 'eval_precision': 0.7907816648446041,
 'eval_recall': 0.8139819950164777,
 'eval_f1': 0.8022141244504296,
 'eval_seqeval_acc': 0.9690970708160253,
 'eval_runtime': 102.9388,
 'eval_samples_per_second': 3.886,
 'eval_steps_per_second': 0.486}

In [33]:
trainer.evaluate(eval_dataset=new_ds_val)

{'eval_loss': 0.09461623430252075,
 'eval_model_preparation_time': 0.0067,
 'eval_precision': 0.8045223660494839,
 'eval_recall': 0.8186062020673558,
 'eval_f1': 0.8115031815552434,
 'eval_seqeval_acc': 0.9660686091574566,
 'eval_runtime': 13.2746,
 'eval_samples_per_second': 3.767,
 'eval_steps_per_second': 0.527}

### Evaluation

In [34]:
ds_test = select_data(split='test', task='both', size=size)

# test
mask_adjustments, new_attention_masks = update_attention_weights(ds_test, ner_attention_weights_equal)
new_ds_test = create_dataset_attention(ds_test, new_attention_masks)

# load hf/pytorch model
# model = LongformerForTokenClassification.from_pretrained(f'{path}/models/{model_name}/model')

In [35]:
trainer.evaluate(eval_dataset=new_ds_test)

{'eval_loss': 0.1123821958899498,
 'eval_model_preparation_time': 0.0067,
 'eval_precision': 0.7678716161452424,
 'eval_recall': 0.7785996055226825,
 'eval_f1': 0.7731984003917408,
 'eval_seqeval_acc': 0.9570762008358619,
 'eval_runtime': 13.2237,
 'eval_samples_per_second': 3.781,
 'eval_steps_per_second': 0.529}

In [36]:
predictions, labels, metrics = trainer.predict(new_ds_test)
print(f"Metrics: {metrics}")
print(predictions[0])
# print(labels[0])

Metrics: {'test_loss': 0.1123821958899498, 'test_model_preparation_time': 0.0067, 'test_precision': 0.7678716161452424, 'test_recall': 0.7785996055226825, 'test_f1': 0.7731984003917408, 'test_seqeval_acc': 0.9570762008358619, 'test_runtime': 13.6384, 'test_samples_per_second': 3.666, 'test_steps_per_second': 0.513}
[[-2.875      -1.2578125   0.19604492 ... -1.0351562  -0.10699463
   2.9765625 ]
 [ 6.4375     -0.6660156  -1.265625   ... -1.4550781  -1.9169922
  -1.1796875 ]
 [ 9.8984375  -1.3964844  -1.8828125  ... -2.         -1.7099609
  -2.0429688 ]
 ...
 [ 9.6796875  -1.2675781  -1.7382812  ... -2.0585938  -1.5175781
  -2.0625    ]
 [ 9.6796875  -1.2675781  -1.7382812  ... -2.0585938  -1.5175781
  -2.0625    ]
 [ 9.6796875  -1.2675781  -1.7382812  ... -2.0585938  -1.5175781
  -2.0625    ]]


In [37]:
np.save(f'{path}/models/{model_name}/predictions.npy', predictions)
# np.save(f'{path}/models/{model_name}/labels.npy', labels)

## Evaluation

In [38]:
def convert_ids_to_labels(pred, true, task):
    """
    Retrieves label prediction from raw predictions then generates y_pred, y_true for seqeval. Converts
    integers into class labels.

    Input:
        pred = raw predictions from model
        true = original labels from dataset
    Output:
        y_pred
        y_true
    """
    if task == 'ner':
        labels = ['O', 'B-PERSON', 'I-PERSON', 'B-CODE', 'I-CODE', 'B-LOC', 'I-LOC', 'B-ORG', 'I-ORG',
        'B-DEM', 'I-DEM', 'B-DATETIME', 'I-DATETIME', 'B-QUANTITY', 'I-QUANTITY', 'B-MISC', 'I-MISC']
    if task == 'mask':
        labels = ['O', 'B-NO_MASK', 'I-NO_MASK', 'B-DIRECT', 'I-DIRECT', 'B-QUASI', 'I-QUASI']

    # create y_pred
    y_pred = [np.argmax(p, axis=1) for p in pred]
    y_pred = [[labels[x] for x in p] for p in y_pred]

    # create y_true
    y_true = [[0 if x == -100 else x for x in sample] for sample in true]
    y_true = [[labels[x] for x in p] for p in y_true]

    return y_pred, y_true

In [39]:
# predictions = np.load(f'{path_pred}/predictions.npy')
# ds = select_data(split='test', task='mask', size=size)
# true_labels = ds['train']['labels']
# true_labels = ds['labels'] # binary
true_labels = new_ds_test['labels'] # pre_attention

y_pred, y_true = convert_ids_to_labels(predictions, true_labels, task='mask')
print('y_pred', [len(i) for i in y_pred])
print('y_true', [len(i) for i in y_true])

y_pred [4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096]
y_true [4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096]


In [40]:
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, zero_division=1)
recall = recall_score(y_true, y_pred, zero_division=1)
f1 = f1_score(y_true, y_pred)

print('accuracy:', precision)
print('precision:', precision)
print('recall:', recall)
print('f1 score:', f1)

report = classification_report(y_true, y_pred)
print(report)

accuracy: 0.7616980221900627
precision: 0.7616980221900627
recall: 0.7785996055226825
f1 score: 0.77005608388198
              precision    recall  f1-score   support

      DIRECT       0.87      0.59      0.70       457
     NO_MASK       0.75      0.47      0.58      1620
       QUASI       0.76      0.92      0.83      4007

   micro avg       0.76      0.78      0.77      6084
   macro avg       0.79      0.66      0.71      6084
weighted avg       0.76      0.78      0.76      6084

